In [42]:
import cv2
import glob
import numpy as np
import numpy
import os
import matplotlib.pyplot as plt

from win32gui import GetWindowText, GetForegroundWindow, GetWindowRect, FindWindow
import pyscreenshot as ImageGrab
import time
import os
from IPython.display import display, clear_output
import datetime
from KartriderKit.Utils.location import locateOnPicture, locateCenterOnPicture
'''
# if the foreground window is as the same as desired_window, then this script will capture your current screen and check current scenario.
# which is faster than capture screenshot using Nox.
'''
WORKPATH = "D:/GameProjects/Kartrider/SmartKartRider"
desired_window = "BlueStacks App Player"
adb_port = 51079
os.chdir(WORKPATH)

In [22]:
target_size = (1280,720)
model_input_shape = (299,299)
first_classes = ['dialog', 'loading', 'others', 'reward']
second_classes = ['idle', 'inQuest']

In [24]:
# The Nox install path

'''
The simulator default port, if you using multiple simulator, you should change the port.
You can check your port number use : os.popen('adb devices').read()
'''
print(os.popen('adb devices').read())
connect = os.popen("adb connect 127.0.0.1:" + str(adb_port)).read()
print(connect)

List of devices attached
127.0.0.1:51079	device
127.0.0.1:65532	offline


already connected to 127.0.0.1:51079



In [25]:
def read_and_resize(path,target_size):
    return cv2.resize(cv2.imread(path),target_size)/255

def crop_img(inputImg, scenario,model_input_shape):
    if scenario == 'reward':
        return inputImg[:,335:940]
    elif scenario == 'dialog':
        return inputImg[460:,]
    elif scenario == 'questArrow':
        return inputImg[460:,]
    elif scenario == 'autoIcon':
        icon = inputImg[632:698,351:416]
        icon = cv2.resize(icon,model_input_shape)
        return icon

In [35]:
os.system('adb shell screencap -p /sdcard/screenshot.png')
os.system('adb pull /sdcard/screenshot.png ./screenshot.png')
im = cv2.imread('./screenshot.png')
im

array([[[51, 50, 49],
        [51, 50, 49],
        [51, 50, 49],
        ...,
        [26, 14, 10],
        [26, 14, 10],
        [26, 14, 10]],

       [[51, 50, 50],
        [51, 50, 50],
        [51, 50, 50],
        ...,
        [26, 14, 10],
        [26, 14, 10],
        [26, 14, 10]],

       [[51, 50, 50],
        [51, 50, 50],
        [51, 50, 50],
        ...,
        [26, 14, 10],
        [26, 14, 10],
        [26, 14, 10]],

       ...,

       [[11,  7, 22],
        [11,  7, 22],
        [12,  7, 23],
        ...,
        [12,  7, 11],
        [12,  7, 11],
        [12,  7, 11]],

       [[11,  7, 21],
        [11,  7, 22],
        [11,  7, 22],
        ...,
        [12,  7, 11],
        [12,  7, 11],
        [12,  7, 11]],

       [[11,  7, 21],
        [11,  7, 21],
        [11,  7, 22],
        ...,
        [12,  7, 11],
        [12,  7, 11],
        [12,  7, 11]]], dtype=uint8)

In [44]:
im = cv2.imread('./screenshot.png')
image = cv2.imread("D:/GameProjects/Kartrider/SmartKartRider/raw_data/US/Button/Close-Window.PNG")
retVal = locateCenterOnPicture(image, im)
retVal

Point(x=1254, y=24)

In [ ]:
run_count = 0
while True:
    clear_output(wait=True)
    run_count += 1
    print(f"Current round count:{run_count}")
    if run_count%500==0:
        #restart adb server and connection after a while
        os.system("adb shell kill-server")
        os.system("adb shell start-server")
        os.popen("adb connect 127.0.0.1:" + str(adb_port))
    print(datetime.datetime.now())
    s = time.time()
    current_window = GetWindowText(GetForegroundWindow())

    # If current_window is not as desired_window `夜神模擬器`, capture screen use adb tools
    if current_window != desired_window:
        try:
            print("ForegroundWindow not BlueStacks, adb screencap is using.")
            os.system('adb shell screencap -p /sdcard/screenshot.png')
            os.system('adb pull /sdcard/screenshot.png ./screenshot.png')
            im = cv2.imread('./screenshot.png')
            os.system('adb shell rm /sdcard/screenshot.png')
            os.remove('./screenshot.png')
        except:
            print("Restart adb service")
            os.system("adb shell kill-server")
            os.system("adb shell start-server")
            os.popen("adb connect 127.0.0.1:" + str(adb_port))
            continue
    else:
        rect = GetWindowRect(FindWindow(None, desired_window))
        im = ImageGrab.grab(bbox=(rect[0],rect[1]+35,rect[2],rect[3]))
        im = np.array(im)
    im = cv2.resize(im,target_size)/255
    current_scenario = check_scenario(im)
    print(f"Computing time : {round(time.time() - s,3)} second.")
    exit

In [ ]:
    if current_scenario == 'dialog':
        print("dialog")
        os.system("adb shell input tap 515 682")
        os.system("adb shell input tap 1377 521")
        time.sleep(2)
    elif current_scenario == 'reward':
        print("reward")
        os.system("adb shell input tap 795 795")
        time.sleep(2)
    elif current_scenario == 'idle':
        print("idle")
        os.system("adb shell input tap 134 294")
        time.sleep(2)
    elif current_scenario == 'inQuest':
        time.sleep(10)
    else:
        time.sleep(2)


## begin game

In [26]:
# os.system("adb shell input tap 640 640") ## begin
# os.system("adb shell input tap 1256 25") ## Close sub window

0

In [ ]:
##s